# Decision Tree

## CART: 分类与回归树

分类回归树是一棵二叉树，且每个非叶子节点都有两个孩子（孩子结点（child node）：结点的子树的根称为该结点的孩子；叶子结点：也叫终端结点，是度为 0 的结点；），所以对于第一棵子树其叶子节点数比非叶子节点数多1。

CART与ID3区别： CART中用于选择变量的不纯性度量是Gini指数； 如果目标变量是标称的，并且是具有两个以上的类别，则CART可能考虑将目标类别合并成两个超类别（双化）； 如果目标变量是连续的，则CART算法找出一组基于树的回归方程来预测目标变量。

https://www.jianshu.com/p/b90a9ce05b28

### CART算法有两步：

#### 决策树生成和剪枝。

- 决策树生成：

递归地构建二叉决策树的过程，基于训练数据集生成决策树，生成的决策树要尽量大；

自上而下从根开始建立节点，在每个节点处要选择一个最好的属性来分裂，使得子节点中的训练集尽量的纯。

不同的算法使用不同的指标来定义"最好"：

- 分类问题，可以选择GINI，双化或有序双化；
- 回归问题，可以使用最小二乘偏差（LSD）或最小绝对偏差（LAD）。

- 决策树剪枝：用验证数据集对已生成的树进行剪枝并选择最优子树，这时损失函数最小作为剪枝的标准。

这里用代价复杂度剪枝 Cost-Complexity Pruning(CCP)



## ID3 算法原理和实现
ID3算法以信息论为基础，其核心是“信息熵”。ID3算法通过计算每个属性的信息增益，认为信息增益高的是好属性，每次划分选取信息增益最高的属性为划分标准，重复这个过程，直至生成一个能完美分类训练样例的决策树。 


<img src='dt.png'>

- 优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据。 
- 缺点：可能会产生过度匹配问题。 
- 适用数据类型：数值型和标称型,**不能处理连续数据**
- **ID3算法只有树的生成，所以该算法生成的树很容易过拟合**，没有加入正则项和剪枝操作。后面的C4.5和CART，以及决策树的剪枝会在详细说明。

# Gradient boosting

Gradient boosting 就是通过加入新的弱学习器，来努力纠正前面所有弱学习器的残差(Loss function)，最终这样多个学习器相加在一起用来进行最终预测，准确率就会比单独的一个要高。之所以称为 Gradient，是因为在**添加新模型时使用了梯度下降算法来最小化的损失**。

前面已经知道，XGBoost 就是对 gradient boosting decision tree 的实现，但是一般来说，gradient boosting 的实现是比较慢的，因为每次都要先构造出一个树并添加到整个模型序列中。

而 XGBoost 的特点就是计算速度快，模型表现好，这两点也正是这个项目的目标。


# XGBOOST原理及缺点

<img src='xgb.png'>

### 原理
对于上面给出的目标函数，我们可以进一步化简,定义树的复杂度

对于f的定义做一下细化，把树拆分成**结构部分q和叶子权重部分w**。下图是一个具体的例子。**结构函数q把输入映射到叶子的索引号上面去**，而w给定了每个索引号对应的**叶子分数**是什么。

https://blog.csdn.net/a819825294/article/details/51206410

<img src='pic.png'>

在这种新的定义下，我们可以把目标函数通过泰勒展开，将公式化简为：

<img src='1.png'>

通过对$w_j$求导等于0，可以得到:$$w_j = -\frac{G_j}{H_j+\lambda}$$

最终化简得到：

<img src='2.png'>

#### g1,h1...叫做“梯度数据”。

## 分裂节点的两种算法：
- 贪心算法
- 近似算法

### 贪心算法：
缺点：
- 在每一次迭代的时候，都需要遍历整个训练数据多次。如果把整个训练数据装进内存则会限制训练数据的大小；
- 如果不装进内存，反复地读写训练数据又会消耗非常大的时间。

<img src='https://upload-images.jianshu.io/upload_images/536604-bc174faf94fb39eb.png'>

# 随机森林，GBDT，XGBoost的对比
**随机森林的集成学习方法是bagging** ，但是和bagging 不同的是bagging只使用bootstrap**有放回的采样样本**，但随机森林即随机采样样本，也随机选择特征，因此**防止过拟合能力更强，降低方差**。

#### 有放回抽样
如果不放回的话，就很容易造成预测偏差，比如样本不均衡时，如果第一次抽样就已经抽出了少的哪类，剩下的那类几乎都是多的那一类了，在此基础上再进行下几轮的抽样和预测训练的话，就跟少的那类的特征没啥关系了，基于此作出的predict值也肯定不准了。

https://zhuanlan.zhihu.com/p/24851814

Bootstrap通过重抽样，可以避免了Cross-Validation造成的样本减少问题，其次，Bootstrap也可以用于创造数据的随机性。比如，我们所熟知的**随机森林算法第一步就是从原始训练数据集中，应用bootstrap方法有放回地随机抽取k个新的自助样本集，并由此构建k棵分类回归树**。

### Bootstrap步骤：
1. 在原有的样本中通过重抽样抽取一定数量（比如100）的新样本，重抽样（Re-sample）的意思就是有放回的抽取，即一个数据有可以被重复抽取超过一次。

2. 基于产生的新样本，计算我们需要估计的统计量。

在这例子中，我们需要估计的统计量是$\alpha$
，那么我们就需要基于新样本的计算样本方差、协方差的值作为$\hat{\sigma _X^2},\hat{\sigma_Y^2}以及\hat{\sigma_{XY}}，然后通过上面公式算出一个\hat{\alpha}$


3. 重复上述步骤n次（一般是n>1000次）。
在这个例子中，通过n次（假设n=1000），我们就可以得到1000个$\alpha_i。也就是\alpha_1,\alpha_2,\cdots,\alpha_{1000}。$



- Bootstrap是现代统计学较为流行的一种统计方法，在小样本时效果很好。通过方差的估计可以构造置信区间等，其运用范围得到进一步延伸。

- 就是一个在自身样本重采样的方法来估计真实分布的问题

- 当我们不知道样本分布的时候，bootstrap方法最有用。

# Bootstrap(自助法)，Bagging，Boosting(提升)

### 集成学习（ensemble learning）
一句话，假设各弱分类器间具有一定差异性（如不同的算法，或相同算法不同参数配置），这会导致生成的分类决策边界不同，也就是说它们在决策时会犯不同的错误。将它们结合后能得到更合理的边界，减少整体错误，实现更好的分类效果。

首先：bagging和boosting都是集成学习（ensemble learning）领域的基本算法

- ### Bagging(bootstrap aggregation)

bagging：
从训练集进行子抽样组成**每个基模型所需要的子训练集**，对所有基模型预测的结果进行综合,产生最终的预测结果,至于为什么叫bootstrap aggregation，因为它抽取训练样本的时候采用的就是bootstrap的方法！

1. 从样本集中用Bootstrap采样选出n个训练样本(放回，因为别的分类器抽训练样本的时候也要用)
<font color='red'> 但是没有进行对特征的抽样，只有对样本的。</font>
2. 在所有属性上，用这n个样本训练分类器（CART or SVM or ...）
3. 重复以上两步m次，就可以得到m个分类器（CART or SVM or ...）
4. 将数据放在这m个分类器上跑，最后**投票**机制(多数服从少数)看到底分到哪一类(分类问题)

#### <font color="red">但是bagging和voting又不一样！！（Titanic那个project里有用，要再研究一下）

- ### Bagging代表算法-RF(随机森林)

1. 用Random(训练样本用Bootstrap方法，选择分离叶子节点用随机选择特征的方式构造一棵决策树(CART)
2. 用1的方法构造很多决策树,每棵决策树都最大可能地进行生长而不进行剪枝，许多决策树构成一片森林，**决策树之间没有联系**
3. 测试数据进入每一棵决策树，每棵树做出自己的判断，然后进行**投票**选出最终所属类别(默认每棵树权重一致)

 ### RF优点：
1. 不容易出现过拟合，因为选择训练样本的时候就不是全部样本。
2. 可以既可以处理属性为离散值的量，比如ID3算法来构造树，也可以处理属性为连续值的量，比如C4.5算法来构造树。
3. 对于高维数据集的处理能力令人兴奋，它可以处理成千上万的输入变量，并确定最重要的变量，因此被认为是一个不错的降维方法。此外，该模型能够输出变量的重要性程度（每个叶子的w），这是一个非常便利的功能。
4. 分类不平衡的情况时，随机森林能够提供平衡数据集误差的有效方法，因为是多个decision boundary的组合，会将少量的样本少的类别也照顾的较好。

### RF缺点：
随机森林在解决回归问题时并没有像它在分类中表现的那么好，这是因为**它并不能给出一个连续型的输出**。当进行回归时，随机森林不能够作出超越训练集数据范围的预测，这可能导致在对某些还有特定噪声的数据进行建模时出现过度拟合。

# Boosting
核心：Boosting是一种**框架算法**，用来**提高弱分类器准确度**的方法，这种方法通过构造一个预测函数序列，然后以一定的方式将他们组合成为一个准确度较高的预测函数，还有就是，Boosting算法更加关注错分的样本，这点和Active Learning的寻找最有价值的训练样本有点遥相呼应的感觉。              

# Boosting算法代表--Adaboost(Adaptive Boosting)

1. 一种迭代算法，针对**同一个训练集**训练**不同的分类器(弱分类器)**
2. 然后进行分类，对于分类正确的样本权值低，**分类错误的样本权值高（通常是边界附近的样本）**，最后的分类器是很多弱分类器的线性叠加（**加权组合**，<font color="red">根据谁作为权重？下边就讲了。</font>），分类器相当简单。实际上就是一个简单的弱分类算法提升(boost)的过程。

https://wizardforcel.gitbooks.io/dm-algo-top10/content/adaboost.html

<img src='ada.png'>

**更新权重的方法：**

该方法只与上一轮样本权重和弱分类器的权重 alpha 值有关。具体计算公式为weighti = weight[i]*exp(-alpha*flag)，可以看出若样本分类正确，则flag为1，exp(-alpha*flag)越小，则新权重越小；若样本分类错误，则flag为-1，exp(-alpha*flag)越大，则新权重会越大。这会使得上一轮分类错误地样本的权重变大，使其在新一轮训练中得到重视。

def calcNewWeight(alpha,ygx, weight, gx, y):
    newWeight = []
    sumWeight = 0
    for i in range(len(weight)):
        flag = 1
        if i  < gx and y[i] != ygx: flag = -1
        if i > gx and y[i] != -ygx: flag = -1
        # 这种方法是双向更新weight，分对的变小，分错的变大。和上边那种分对了不变的算法不一样。
        weighti = weight[i]*exp(-alpha*flag)
        newWeight.append(weighti)
        sumWeight += weighti
    newWeight = newWeight / sumWeight
 
    return newWeight

<img src='ada1.png'>

- 迭代终止条件
不断重复1,2,3步骤，直到达到终止条件为止。也不可能让所有点都被分对，y=h，所以终止条件是强分类器的错误率低于最低错误率阈值或达到最大迭代次数。

#### AdaBoost 不止适用于分类模型，也可以用来训练回归模型。这需要将弱分类器替换成回归模型，并改动损失函数。

### Adaboost优点
1. 可以使用各种方法构造子分类器，Adaboost算法提供的是框架
2. 简单，不用做特征筛选
3. 相比较于RF，更不用担心过拟合问题（Bella thinks：因为是多个算法组合，所以相对于一个模型，多次取样的RF来说，更不会出现过拟合问题。）

### Adaboost缺点
1. 从wiki上介绍的来看，adaboost对于噪音数据和异常数据是十分敏感的。Boosting方法本身**对噪声点异常点很敏感**，因此在每次迭代时候会给噪声点较大的权重，这不是我们系统所期望的。<br>但是对比样本极不均衡的点，或者是更加在意某一类点，觉得将少数点全部分对要纳入一些异常点更重要的case，Adaboost就很合适，比如对customer churn，多纳入一些本来不会churn但是却被预测为churn的datapoint并不很在意。
2. 运行速度慢，凡是涉及迭代的基本上都无法采用并行计算，Adaboost是一种"串行"算法.
3. 所以GBDT(Gradient Boosting Decision Tree)也非常慢。
4. AdaBoost 算法只直接支持二分类，遇到多分类的情况，需要借助 one-versus-rest 的思想来训练多分类模型。关于 one-verus-rest 的细节可以参考本系列第一篇文章 SVM。


# conclusion：
1. Bagging： 树"并行"生成 ,如RF;Boosting：树"串行"生成,如Adaboost，第一个模型在train dataset上预测完数据之后，针对分错的样本提升weight，再在下一个模型上着重分类。所以是串行结构。

2. boosting中的基模型为弱模型，而RF中的基树是强模型(大多数情况)

3. boosting重采样的不是样本，而是样本的分布，每次迭代之后，样本的分布会发生变化，也就是被分错的样本会更多的出现在下一次训练集中

4. AdaBoost 算法只直接支持二分类，遇到多分类的情况，需要借助 one-versus-rest 的思想来训练多分类模型。关于 one-verus-rest 的细节可以参考本系列第一篇文章 SVM。


另一种分类方法：Adaboost基于树模型，弱分类器每次都是树模型，每次训练的时候样本都是一样的，只是特征选取上不一样，因为每次迭代完数据点的权重会更新，所以每次训练时选取切分的特征也不一样：还附了代码。
https://www.ibm.com/developerworks/cn/analytics/library/machine-learning-hands-on6-adaboost/index.html

# 面试整理：RF、GBDT、XGBoost

# Random Forest

提到随机森林，就不得不提Bagging，Bagging可以简单的理解为：放回抽样，**多数表决（分类）或简单平均（回归）**,同时Bagging的基学习器之间属于**并列生成，并行**，不存在强依赖关系。 

Random Forest（随机森林）是Bagging的扩展变体，它在以决策树 为基学习器构建Bagging集成的基础上，进一步在决策树的训练过程中**引入了随机特征选择**，因此可以概括RF包括四个部分：

1. 随机选择样本（放回抽样）；
2. 随机选择特征；
3. 构建决策树；
4. 随机森林投票（平均）。 
　
 随机选择样本和Bagging相同，随机选择特征是指在树的构建中，会从样本集的特征集合中随机选择部分特征，然后再从这个子集中选择最优的属 性用于划分，这种随机性导致随机森林的偏差会有稍微的增加（相比于单棵不随机树），但是由于随机森林的‘平均’特性，会使得它的方差减小，而且方差的减小补偿了偏差的增大，因此总体而言是更好的模型。 

在构建决策树的时候，RF的每棵决策树都**最大可能的进行生长而不进行剪枝,因为只是部分样本，不不太可能过拟合**；在对预测输出进行结合时，RF通常对分类问题使用简单投票法，回归任务使用简单平均法。 

**RF的重要特性是不用对其进行交叉验证或者使用一个独立的测试集获得无偏估计**，它可以在内部进行评估，也就是说在生成的过程中可以对误差进行无偏估计，由于每个基学习器只使用了训练集中约63.2%的样本，剩下约36.8%的样本可用做验证集来对其泛化性能进行“包外估计”。 

RF和Bagging对比：RF的起始性能较差，特别当只有一个基学习器时，随着学习器数目增多，随机森林通常会收敛到更低的泛化误差。随机森林的训练效率也会高于Bagging，因为**在单个决策树的构建中，Bagging使用的是‘确定性’决策树，在选择特征划分结点时，要对所有的特征进行考虑，而随机森林使用的是‘随机性’特征数，只需考虑特征的子集**。


随机森林顾名思义，是用随机的方式建立一个森林，森林里面有很多的决策树组成，随机森林的每一棵决策树之间是没有关联的。在得到森林之后，当有一个新的输入样本进入的时候，就让森林中的每一棵决策树分别进行一下判断，看看这个样本应该属于哪一类（对于分类算法），**然后看看哪一类被选择最多，就预测这个样本为那一类。**

由于之前的两个随机采样的过程保证了随机性，所以**不剪枝**，也不会出现over-fitting。

### 优缺点
随机森林的优点较多，简单总结：
1、在数据集上表现良好，相对于其他算法有较大的优势（训练速度、预测准确度）；

2、能够处理很高维的数据，并且不用特征选择，而且在训练完后，给出特征的重要性；

3、容易做成并行化方法。 

RF的缺点：在噪声较大的分类或者回归问题上回过拟合。**虽然在每棵决策树上不用剪枝，不担心过拟合，但是因为也正因为每棵决策树都不剪枝，合起来就会出现过拟合的结果**。


# GBDT
提GBDT之前，谈一下Boosting，Boosting是一种与Bagging很类似的技术。不论是Boosting还是Bagging，所使用的**多个分类器类型都是一致的**，比如bagging的代表RF中的多个分类器就都是决策树，boosting也是，里边可以套任意的基分类器。

但是Boosting当中，不同的分类器是通过串行训练而获得的，每个新分类器都根据已训练的分类器的性能来进行训练。Boosting是通过关注被已有分类器错分的那些数据来获得新的分类器。 

由于Boosting分类的结果是基于所有分类器的加权求和结果的，因此Boosting与Bagging不太一样，**Bagging中的分类器权值是一样的，而Boosting中的分类器权重并不相等，每个权重代表对应的分类器在上一轮迭代中的成功度。**


## 原理
　　GBDT与传统的Boosting区别较大，它的每一次计算都是为了减少上一次的残差，而为了消除残差，我们可以**在残差减小的梯度方向上建立模型,所以说，在GradientBoost中，每个新的模型的建立是为了使得之前的模型的残差往梯度下降的方法，与传统的Boosting（比如Adaboost）中关注正确错误的样本加权有着很大的区别。 
  
　　在GradientBoosting算法中，关键就是**利用损失函数的负梯度方向在当前模型的值作为残差的近似值，进而拟合一棵CART回归树，回归哦！**。
  
　　GBDT的会**累加所有树的结果，而这种累加是无法通过分类完成的，因此GBDT的树都是CART回归树，而不是分类树（尽管GBDT调整后也可以用于分类但不代表GBDT的树为分类树）。**


## 优缺点
GBDT的性能在RF的基础上又有一步提升，因此其优点也很明显，
  1. 它能灵活的处理各种类型的数据；
  2. 在相对较少的调参时间下，预测的准确度较高。 

当然由于它是Boosting，因此基学习器之前存在串行关系，难以并行训练数据

## GBDT也叫GBRT
全名gradient boosted decision tree,是一種新的思想。也是由很多棵決策樹**（迴歸樹）**組成，在每訓練一棵樹時，它的訓練數據是已經訓練好的模型（前面訓練好的樹）的**殘差（目標函數的梯度）**。（决策树的目标/损失函数是GINI系数 OR 信息熵，但这个是残差/MSE），也就是用迴歸樹來優化目標函數。詳細的看這裏的文章，代碼看另外一個作者的。

https://blog.csdn.net/w28971023/article/details/8240756

# GBDT（MART） 迭代决策树入门教程
#### GBDT(Gradient Boosting Decision Tree) 又叫 MART（Multiple Additive Regression Tree)，是一种迭代的决策树算法，该算法由多棵决策树组成，所有树的结论累加起来做最终答案。它在被提出之初就和SVM一起被认为是泛化能力（generalization)较强的算法。近些年更因为被用于搜索排序的机器学习模型而引起大家关注。

#### GBDT主要由三个概念组成：Regression Decistion Tree（即DT)，Gradient Boosting（即GB)，Shrinkage (算法的一个重要演进分枝，目前大部分源码都按该版本实现）。搞定这三个概念后就能明白GBDT是如何工作的，要继续理解它如何用于搜索排序则需要额外理解RankNet概念，之后便功德圆满。下文将逐个碎片介绍，最终把整张图拼出来。

 



提起决策树（DT, Decision Tree) 绝大部分人首先想到的就是C4.5分类决策树。但如果一开始就把GBDT中的树想成分类树，那就是一条歪路走到黑，一路各种坑，最终摔得都要咯血了还是一头雾水说的就是LZ自己啊有木有。咳嗯，所以说千万不要以为GBDT是很多棵分类树。决策树分为两大类，回归树和分类树。前者用于预测实数值，如明天的温度、用户的年龄、网页的相关程度；后者用于分类标签值，如晴天/阴天/雾/雨、用户性别、网页是否是垃圾页面。这里要强调的是，前者的结果加减是有意义的，如10岁+5岁-3岁=12岁，后者则无意义，如男+男+女=到底是男是女？ **GBDT的核心在于累加所有树的结果作为最终结果**，就像前面对年龄的累加（-3是加负3），而分类树的结果显然是没办法累加的，所以**GBDT中的树都是回归树，不是分类树**，（而且只有具体数值才能gradient吧）这点对理解GBDT相当重要（尽管GBDT调整后也可用于分类但不代表GBDT的树是分类树）。那么回归树是如何工作的呢？

回归树总体流程也是类似，不过**在每个节点（不一定是叶子节点）都会得一个预测值**，以年龄为例，该预测值等于属于这个节点的所有人年龄的平均值。分枝时穷举每一个feature的每个阈值找最好的分割点，但衡量最好的标准不再是最大熵，而是最小化均方差--即（每个人的年龄-预测年龄）^2 的总和 / N，或者说是每个人的预测误差平方和 除以 N。这很好理解，被预测出错的人数越多，错的越离谱，均方差就越大，通过最小化均方差能够找到最靠谱的分枝依据。分枝直到每个叶子节点上人的年龄都唯一（这太难了）或者达到预设的终止条件（如叶子个数上限），若最终叶子节点上人的年龄不唯一，则以该节点上所有人的平均年龄做为该叶子节点的预测年龄。若还不明白可以Google "Regression Tree"，或阅读本文的第一篇论文中Regression Tree部分。

Boosting，迭代，即通过迭代多棵树来共同决策。这怎么实现呢？难道是每棵树独立训练一遍，比如A这个人，第一棵树认为是10岁，第二棵树认为是0岁，第三棵树认为是20岁，我们就取平均值10岁做最终结论？--当然不是！且不说这是投票方法并不是GBDT，只要训练集不变，独立训练三次的三棵树必定完全相同，这样做完全没有意义。之前说过，GBDT是把所有树的结论累加起来做最终结论的，所以可以想到每棵树的结论并不是年龄本身，而是年龄的一个累加量。**GBDT的核心就在于，每一棵树学的是之前所有树结论和的残差，这个残差就是一个加预测值后能得真实值的累加量。**比如A的真实年龄是18岁，但第一棵树的预测年龄是12岁，差了6岁，即残差为6岁。那么在第二棵树里我们把A的年龄设为6岁去学习，如果第二棵树真的能把A分到6岁的叶子节点，那累加两棵树的结论就是A的真实年龄；如果第二棵树的结论是5岁，则A仍然存在1岁的残差，第三棵树里A的年龄就变成1岁，继续学。这就是Gradient Boosting在GBDT中的意义，简单吧。

有个例子，超清楚。

在第一棵树分枝和图1一样，由于A,B年龄较为相近，C,D年龄较为相近，他们被分为两拨，每拨用平均年龄作为预测值。此时计算残差，所以A的残差就是16-15=1（注意，**A的预测值是指前面所有树累加的和，而在一棵树里是没有残差的，只有在下一棵树中，才用残差来继续建树。**，这里前面只有一棵树所以直接是15，如果还有树则需要都累加起来作为A的预测值）。进而得到A,B,C,D的残差分别为-1,1，-1,1。然后我们拿残差替代A,B,C,D的原值，到第二棵树去学习，如果我们的预测值和它们的残差相等，则只需把第二棵树的结论累加到第一棵树上就能得到真实年龄了。这里的数据显然是我可以做的，第二棵树只有两个值1和-1，直接分成两个节点。此时所有人的残差都是0，即每个人都得到了真实的预测值。

那么哪里体现了Gradient呢？其实回到第一棵树结束时想一想，无论此时的cost function是什么，是均方差还是均差，只要它以误差作为衡量标准，**残差向量(-1, 1, -1, 1)都是它的全局最优方向，这就是Gradient**。

#### 既然图1和图2 最终效果相同，为何还需要GBDT呢？

答案是过拟合。过拟合是指为了让训练集精度更高，学到了很多”仅在训练集上成立的规律“，导致换一个数据集当前规律就不适用了。其实只要允许一棵树的叶子节点足够多，训练集总是能训练到100%准确率的（大不了最后一个叶子上只有一个instance)。在训练精度和实际精度（或测试精度）之间，后者才是我们想要真正得到的。

我们发现**图1为了达到100%精度使用了3个feature**（上网时长、时段、网购金额），其中分枝“上网时长>1.1h” 很显然已经过拟合了，这个数据集上A,B也许恰好A每天上网1.09h, B上网1.05小时，但用上网时间是不是>1.1小时来判断所有人的年龄很显然是有悖常识的；

相对来说**图2的boosting虽然用了两棵树 ，但其实只用了2个feature就搞定了**，后一个feature是问答比例，显然图2的依据更靠谱。（当然，这里是LZ故意做的数据，所以才能靠谱得如此狗血。实际中靠谱不靠谱总是相对的） Boosting的最大好处在于，**每一步的残差计算其实变相地增大了分错instance的权重，而已经分对的instance则都趋向于0。这样后面的树就能越来越专注那些前面被分错的instance**。就像我们做互联网，总是先解决60%用户的需求凑合着，再解决35%用户的需求，最后才关注那5%人的需求，这样就能逐渐把产品做好，因为不同类型用户需求可能完全不同，需要分别独立分析。如果反过来做，或者刚上来就一定要做到尽善尽美，往往最终会竹篮打水一场空。

到目前为止，我们的确**没有用到求导的Gradient**。在当前版本GBDT描述中，的确没有用到Gradient，**该版本用残差作为全局最优的绝对方向，并不需要Gradient求解.**

## Shrinkage 

Shrinkage（缩减）的思想认为，每次走一小步逐渐逼近结果的效果，要比每次迈一大步很快逼近结果的方式更容易避免过拟合。即它不完全信任每一个棵残差树，它认为每棵树只学到了真理的一小部分，**累加的时候只累加一小部分，通过多学几棵树弥补不足**。用方程来看更清晰，即

没用Shrinkage时：（yi表示第i棵树上y的预测值， y(1~i)表示前i棵树y的综合预测值）

$y_{i+1} = 残差(y_1 ~ y_i)， 其中： 残差(y_1 ~ y_i) =  y_{真实值} - y_{1 ~ i}$

y(1 ~ i) = SUM(y1, ..., yi)

Shrinkage不改变第一个方程，只把第二个方程改为： 

y(1 ~ i) = y(1 ~ i-1) + step * yi

即Shrinkage仍然以残差作为学习目标，但对于残差学习出来的结果，只累加一小部分（step*残差）逐步逼近目标，step(类似学习率)一般都比较小，如0.01~0.001（注意该step非gradient的step），**导致各个树的残差是渐变的而不是陡变的**。直觉上这也很好理解，不像直接用残差一步修复误差，而是只修复一点点，其实就是把大步切成了很多小步。本质上，**Shrinkage为每棵树设置了一个weight，累加时要乘以这个weight，但和Gradient并没有关系**。这个weight就是step。就像Adaboost一样，Shrinkage能减少过拟合发生也是经验证明的，目前还没有看到从理论的证明。

<img src='http://i.imgur.com/bZ6VIzH.png'>

## GBDT的适用范围

该版本GBDT几乎可用于所有回归问题（线性/非线性），相对logistic regression仅能用于线性回归，GBDT的适用面非常广。**亦可用于二分类问题（设定阈值，大于阈值为正例，反之为负例）。但这并不意味着GBDT是分类树。**

GBDT也叫GBRT
全名gradient boosted decision tree,是一種新的思想。也是由很多棵決策樹（迴歸樹）組成，在每訓練一棵樹時，它的訓練數據是已經訓練好的模型（前面訓練好的樹）的殘差（目標函數的梯度）。也就是用迴歸樹來優化目標函數。詳細的看這裏的文章，代碼看另外一個作者的。

### GBDT 特点：
GBDT 是一个加性回归模型，通过 boosting 迭代的构造一组弱学习器，**相对LR的优势如不需要做特征的归一化，自动进行特征选择，模型可解释性较好，可以适应多种损失函数如 SquareLoss，LogLoss 等等**。

但作为非线性模型，其相对线性模型的缺点也是显然的：**boosting 是个串行的过程，不能并行化，计算复杂度较高，同时其不太适合高维稀疏特征**，通常采用稠密的数值特征如点击率预估中的 COEC。—引自xgboost导读与实战


## 搜索引擎排序应用 RankNet

#### GDBT中所的DT(Decision Tree)树都是回归树而不是分类树,所以GDBT也叫MART(Multiple Additive Regression Tree)、GBRT(Gradient Boost Regression Tree)。下面主要叙述回归问题的提升树。 

注意：
<br>这里的拟合残差指根据回归树的生成算法步骤，用平方误差最小化准则求解每个单元上的最优输出值时，本来应该取输出变量Y与预测值的平方差，在提升树使用前向分步算法到下一个决策树学习的时候，可以**把当前模型拟合数据的残差作为下一个模型学习的输出变量Y’,通过学习每一轮的学习误差(残差)，到下一轮拟合的误差会越来越小，最后将模型输出累加起来，就可以得到最优的输出结果**。 



m: 第m步,第m棵数

<img src='http://i.imgur.com/imJx56v.png'>

#### !!!所有树的累加和。
<img src='http://i.imgur.com/hhbZz3h.png'>

### 案例！

<img src='http://i.imgur.com/NqRfj0q.png'>

# 梯度提升/迭代(Gradient Boosting)

提升树利用加法模型与前向分步算法实现学习的优化过程。当损失函数是平方损失和指数损失函数时，每一步的优化是简单的（在案例8.2中，平方损失误差）。但对于一般的损失函数，每一步的优化并不是那么容易，针对这一问题，Freidman提出了梯度提升(gradient boosting)算法。这是利用最速下降法的近似方法，关键在于**利用损失函数的负梯度在当前模型的值** :

<img src='http://i.imgur.com/j64fQ8Y.png'>

作为回归问题提升树算法中的**残差的近似值，拟合一个回归树**。

**注意：即使这里用了求导，但是其实跟上边那个算出f1，T1再用y相减是一样的。因为平方差的导数就是残差。**

<img src='http://i.imgur.com/2Oy8p6c.jpg'>

# XGBoost
## 原理
　　XGBoost的性能在GBDT上又有一步提升，而其性能也能通过各种比赛管窥一二。坊间对XGBoost最大的认知在于其能够**自动地运用CPU的多线程进行并行计算，同时在算法精度上也进行了精度的提高**。<br> 
　　由于GBDT在合理的参数设置下，往往要生成一定数量的树才能达到令人满意的准确率，在数据集较复杂时，模型可能需要几千次迭代运算。但是XGBoost利用并行的CPU更好的解决了这个问题。 <br>
　　其实XGBoost和GBDT的差别也较大，这一点也同样体现在其性能表现上，详见XGBoost与GBDT的区别。<br>

## GBDT和XGBoost区别
1. 传统的GBDT以CART树作为基学习器，XGBoost还支持线性分类器，这个时候XGBoost相当于L1和L2正则化的逻辑回归（分类）或者线性回归（回归）；
- **传统的GBDT在优化的时候只用到一阶导数信息(残差)，XGBoost则对代价函数进行了二阶泰勒展开，得到一阶和二阶导数**；
- XGBoost在**代价函数中加入了正则项，用于控制模型的复杂度**。从权衡方差偏差来看，它降低了模型的方差，使学习出来的模型更加简单，**防止过拟合**，这也是XGBoost优于传统GBDT的一个特性；
- shrinkage（缩减），相当于学习速率（XGBoost中的eta）。XGBoost在进行完一次迭代时，会将叶子节点的权值乘上该系数，主要是为了削弱每棵树的影响，让后面有更大的学习空间。（GBDT也有学习速率）；
- 列抽样。XGBoost借鉴了随机森林的做法，支持**列抽样**（抽取特征。行抽样是抽取样本），不仅防止过 拟合，还能减少计算；
- 对缺失值的处理。对于特征的值有缺失的样本，XGBoost还可以自动 学习出它的分裂方向；**就不用手工填上特征值了，因为手工也是用RF，boost等这类以随机抽样为基础的算法进行填充。**
- XGBoost工具支持并行。Boosting不是一种串行的结构吗?怎么并行 的？注意XGBoost的并行不是tree粒度的并行，XGBoost也是一次迭代完才能进行下一次迭代的（第t次迭代的代价函数里包含了前面t-1次迭代的预测值）。**XGBoost的并行是在特征粒度上的**。我们知道，决策树的学习最耗时的一个步骤就是对特征的值进行排序（因为要确定最佳分割点），XGBoost在训练之前，预先对数据进行了排序，然后保存为block结构，后面的迭代 中重复地使用这个结构，大大减小计算量。这个block结构也使得并行成为了可能，在进行节点的分裂时，需要计算每个特征的增益，最终选增益最大的那个特征去做分裂，那么**各个特征的增益计算就可以开多线程进行**。
- 可并行的**近似直方图算法**。树节点在进行分裂时，我们需要计算每个特征的每个分割点对应的增益，即用贪心法枚举所有可能的分割点。当数据无法一次载入内存或者在分布式情况下，**贪心算法**效率就会变得很低，所以xgboost还提出了一种可并行的近似直方图算法，用于高效地生成候选的分割点。xgboost的目标函数如下：


9. Shrinkage（缩减），相当于学习速率（xgboost中的eta）。xgboost在进行完一次迭代后，会将叶子节点的权重乘上该系数，主要是为了削弱每棵树的影响，让后面有更大的学习空间。实际应用中，一般把eta设置得小一点，然后迭代次数设置得大一点。（补充：传统GBDT的实现也有学习速率，shrinkage） 

10. 允许使用column(feature) sampling来防止过拟合，借鉴了Random Forest的思想，sklearn里的**gbm**好像也有类似实现。
4. 实现了一种**分裂节点寻找**的近似算法，用于加速和减小内存消耗。
5. 节点分裂算法能自动利用特征的**稀疏性**。<font color='red'>这里还不太懂诶。</font>
6. data事先排好序并以block的形式存储，利于并行计算
7. penalty function Omega主要是对树的叶子数和叶子分数做惩罚，这点确保了树的简单性。。
8. 支持分布式计算可以运行在MPI，YARN上，得益于底层支持容错的分布式通信框架rabit。


### 这个推导写得好：

<img src='1.png'>

#### !!! 函数值误差是针对样本的，复杂度是针对叶子结点的，化简：把样本也化为针对叶子结点进行遍历的。

<img src='2.png'>

### 关于预剪枝的公式含义

<img src='1.png'>

<img src='2.png'>

<img src='3.png'>

这个公式形式上**跟ID3算法、CART算法是一致的**，都是用分裂后的某种值减去分裂前的某种值，从而得到增益。

为了限制树的生长，我们可以加入阈值，当增益大于阈值时才让节点分裂，**上式中的gamma即阈值，它是正则项里叶子节点数T的系数，所以xgboost在优化目标函数(求信息增益Gain的时候)的同时相当于做了预剪枝**。

另外，上式中还有一个系数lambda，是正则项里leaf score的L2模平方的系数，对**leaf score做了平滑，也起到了防止过拟合的作用，这个是传统GBDT里不具备的特性**。传统的GBDT虽然也有剪枝，但是是都算好了之后才剪的。在正则项也只考虑了叶子结点的个数，没有加入L2正则。


#### 基学习器可选为： 
xgboost的弱分类器有三种gbtree, gblinear or dart. 
第一个和第三个都是树。 
第二个是逻辑回归，但是本质它并没有被boost，可以看作只是普通的sgd classifier。

### XGBoost注意事项：
- 多类别分类时，类别需要从0开始编码
- Watchlist不会影响模型训练。
- XGBoost的特征重要性是如何得到的？某个特征的重要性（feature score），等于它被选中为树节点分裂特征的次数的和，比如特征A在第一次迭代中（即第一棵树）被选中了1次去分裂树节点，在第二次迭代被选中2次…..那么最终特征A的feature score就是 1+2+….

# Light GBM 

- lightGBM：基于决策树算法的分布式梯度提升框架。

- Lgb与xgBoost的区别：xgBoost使用的是pre-sorted算法（对所有特征都按照特征的数值进行预排序，在遍历分割点的时候用O(data)的代价找到一个特征上的最好分割点），能够更精确的找到数据分隔点；LightGBM使用的是**histogram算法，占用的内存更低，数据分隔的复杂度更低**。

- 决策树生长策略上：XGBoost采用的是**level-wise生长策略，能够同时分裂同一层的叶子，从而进行多线程优化，不容易过拟合**；但不加区分的对待同一层的叶子，带来了很多没必要的开销（因为实际上很多叶子的分裂增益较低，没必要进行搜索和分裂）；

- LightGBM采用**leaf-wise生长策略，每次从当前所有叶子中找到分裂增益最大（一般也是数据量最大）的一个叶子，然后分裂，如此循环；但会生长出比较深的决策树**，产生过拟合（因此 LightGBM 在leaf-wise之上增加了一个最大深度的限制，在保证高效率的同时防止过拟合）。另一个比较巧妙的优化是**histogram 做差加速**。一个容易观察到的现象：一个叶子的直方图可以由它的父亲节点的直方图与它兄弟的直方图做差得到。
